# Metric and Topological Spatial Queries with Shapely

This notebook tests some simple feature creation and topological relationship testing with python libraries [shapely](http://toblerity.org/shapely/) and [pyproj](http://jswhit.github.io/pyproj/).
We also use GDAL/OGR to convert WKT to geojson to plot with folium.  As we mentioned before, GDAL/OGR are libraries for importing, exporting and transforming between different geospatial data formats.  This package includes functions for accessing Proj4, a library for defining and transforming coordinate reference systems.  Pyproj is another interface to Proj4 with a bit simpler syntax.  Shapely is a library for creating, transforming and analyzing vector geometries - points, lines and polygons. Shapely is based on the widely used [GEOS](https://trac.osgeo.org/geos/)  library.  GDAL/OGR, Proj4, and GEOS are three pillars of many open source geospatial software tools and libraries.


## Homework 4 Instructions

### What to Submit
- Do your work in this notebook. Add markdown or code cells if needed to input your code or responses to questions.
- Save your work frequently as you go along.
- When you are done download your work as a both an ipython notebook and as a PDF file. Zip the two together and give the file a name like yourlastname_hw3.zip
- Upload your zip file on bCourses as your submission for this assignment.
- Get started before the due date so you can email me your questions (pfrontiera@berkeley.edu).
- Due date: ** Monday, March 7, 2016 at 4pm **

In [1]:
# Import libraries
from datascience import *
import json
import folium
import shapely
from shapely import wkt
import numpy as np
from shapely.geometry import Polygon, Point, LineString, shape
import math

In [20]:
# Import GDAL/OGR and define error handling function
from osgeo import ogr, osr, gdal
#from osgeo import ogr, osr, gdal

# Enable GDAL/OGR exceptions
gdal.UseExceptions()

# If gdal is having trouble finding the *.csv files 
# that contain the Proj definitions, set the path explicity
gdal.SetConfigOption('GDAL_DATA','/opt/conda/share/gdal')
# GDAL error handler function so we get more descriptive and helpful error messages
# Source: The Python GDAL/OGR Cookbook
def gdal_error_handler(err_class, err_num, err_msg):
    errtype = {
            gdal.CE_None:'None',
            gdal.CE_Debug:'Debug',
            gdal.CE_Warning:'Warning',
            gdal.CE_Failure:'Failure',
            gdal.CE_Fatal:'Fatal'
    }
    err_msg = err_msg.replace('\n',' ')
    err_class = errtype.get(err_class, 'None')
    print('Error Number: %s' % (err_num))
    print('Error Type: %s' % (err_class))
    print('Error Message: %s' % (err_msg))

gdal.PushErrorHandler(gdal_error_handler)

0

## Part 1. Getting Started
Take a look at the first page of the Shapely Manual at http://toblerity.org/shapely/manual.html.  As it states, Shapely can be used to create vector geometeries such as Points, LineStrings and Polygons and explore topological relationshops between geometries.  Let's warm up by trying the following commands. Uncomment one line at a time and press 'Shift'+'Enter' to execute.
    

In [3]:
# Shapely warm-up commands
# Note:
# Shapely does not support coordinate system transformations. 
# All operations on two or more geometries presume that the geometeries exist in the same Cartesian plane.

a = Point(1, 1).buffer(1.5)    # Line 1
#a
#b = Point(3, 1).buffer(1.5)   # Line 2
#b
#c = a.union(b)                # Line 3
#c
#d = a.intersection(b)         # Line 4
#d
#e = a.intersects(b)           # Line 5
#e
#f = a.contains(b)             # Line 6
#f
#g = a.touches(b)              # Line 7
#g
#h = a.area                    # Line 8
#h
#i = a.centroid                # Line 9
#i
#print(i)
#j = a.buffer(5).contains(b)   # Line 10
#j
#k = Point(6, 6).buffer(1)     
#a.distance(k)                 # Line 11

## Question 1

- 1.1  What lines above are geometry constructors? In other words, they create a new geometry?
- 1.2  What lines above are metric calculations? What do they return?
- 1.3  What lines above are topological queries? What do they return?
- 1.4  Why does a contain b in Line 10 but not in Line 6?

## Question 1 - Double click on this cell to input your answers below.
- 1.1
- 1.2
- 1.3
- 1.4

## Question 2
In the cell below create a Shapely Point, a Shapely LineString, and a Shapely Polygon.
Then write code to answer the questions noted in the comments.

In [4]:
mypoint =     # create a point at 4,4
myline =      # create a line with 3 points that ends at the point
mypolygon =   # create a polygon that intesects the line

# What is the length of your line?
# what is the centroid of your polygon?
# What is the area of your line?
# Does the point touch the line?
# Does the polygon contain the point?
# Does the line intersect the polygon?
# What type of geometry to you get if you buffer the line?


SyntaxError: invalid syntax (<ipython-input-4-94dc7eea5c62>, line 1)

## Part 2. Working with Real Data

Let's read in some data in CSV files that have vector geometry in WKT format.  The coordinates for all of these data are WGS84.  We will be working with
- California Counties
- The boundary of the State of Nevada
- The boundary of Yosemite National Park

In [5]:
calcounties = Table.read_table("./ca_counties_wkt.csv")
calcounties.relabel('NAMELSAD','NAME')
calcounties

WKT,STATEFP,COUNTYFP,NAME,ALAND,AWATER
"MULTIPOLYGON (((-120.655585 39.69356,-120.654227 39.7066 ...",6,91,Sierra County,2468686345,23299112
"MULTIPOLYGON (((-121.188571 38.714308,-121.151857 38.711 ...",6,67,Sacramento County,2499176690,76080730
"MULTIPOLYGON (((-120.581897 34.098557,-120.582264 34.107 ...",6,83,Santa Barbara County,7083926262,2729818040
"MULTIPOLYGON (((-120.630933 38.3411,-120.631795 38.34602 ...",6,9,Calaveras County,2641819811,43808748
"MULTIPOLYGON (((-119.636302 33.27304,-119.63382 33.28932 ...",6,111,Ventura County,4773381212,945947858
"MULTIPOLYGON (((-118.667602 33.477489,-118.666719 33.493 ...",6,37,Los Angeles County,10510365728,1794809423
"MULTIPOLYGON (((-122.93506 38.313952,-122.939709 38.3108 ...",6,97,Sonoma County,4081401375,497545782
"MULTIPOLYGON (((-119.958925 36.255468,-119.959227 36.400 ...",6,31,Kings County,3598582246,5468555
"MULTIPOLYGON (((-117.437426 33.17953,-117.467871 33.2124 ...",6,73,San Diego County,10895213672,826296339
"MULTIPOLYGON (((-121.065436 39.006533,-121.061653 39.010 ...",6,61,Placer County,3644346108,246376805


In [6]:
nevada = Table.read_table("./nevada_wkt.csv")
nevada

WKT,FID_,STATE_NAME,SUB_REGION,STATE_ABBR,POP2010,POP10_SQMI,POP2012,POP12_SQMI,WHITE,BLACK,AMERI_ES,ASIAN,HAWN_PI,HISPANIC,OTHER,MULT_RACE,MALES,FEMALES,AGE_UNDER5,AGE_5_9,AGE_10_14,AGE_15_19,AGE_20_24,AGE_25_34,AGE_35_44,AGE_45_54,AGE_55_64,AGE_65_74,AGE_75_84,AGE_85_UP,MED_AGE,MED_AGE_M,MED_AGE_F,HOUSEHOLDS,AVE_HH_SZ,HSEHLD_1_M,HSEHLD_1_F,MARHH_CHD,MARHH_NO_C,MHH_CHILD,FHH_CHILD,FAMILIES,AVE_FAM_SZ,HSE_UNITS,VACANT,OWNER_OCC,RENTER_OCC,NO_FARMS07,AVG_SIZE07,CROP_ACR07,AVG_SALE07,SQMI
"POLYGON ((-119.152437626810652 38.411795749284181,-119.3 ...",0,Nevada,Mountain,NV,2700551,24.4,2757217,24.9383,1786688,218626,32062,195436,16871,716501,324793,126075,1363616,1336935,187478,183077,183173,182600,177509,387286,383043,376527,315499,197781,96391,30187,36.3,35.8,36.8,1006250,2.65,134121,124288,212405,250104,39193,83426,656621,3.2,1173814,167564,591480,414770,3131,1873,753718,163.93,110561


In [7]:
yosemite = Table.read_table("./yosemite_wkt.csv")
yosemite

WKT,OBJECTID,UNIT_TYPE,STATE,REGION,UNIT_CODE,UNIT_NAME,DATE_EDIT,GIS_NOTES,CREATED_BY,METADATA,PARKNAME,Shape_Leng,Shape_Area
"POLYGON ((-119.695877021274086 37.609881005066654,-119.6 ...",51,National Park,CA,PW,YOSE,Yosemite National Park,2012/11/13,Lands - http://landsnet.nps.gov/tractsnet/documents/YOSE ...,Lands,http://nrdata.nps.gov/programs/Lands/YOSE_metadata.xml,Yosemite,291787,3.01648e+09


Spatial Analysis begins with asking questions about spatial metrics, or measurements, and relationships. Some questions we can ask these data are:
- In which California counties is Yosemite National Park?
- What is the total park area within each of these counties?
- What California counties border Nevada?

In order to answer these questions with Shapely, we need to convert our geometries into shapely geometries.  Let's take a look at how we might do that with one California county.

In [8]:
# A function to see if two geometries are neigbors - defined by having intersecting geometries
def areTheyNeighbors(search_geom_str,target_geom_str):
    search_geom = wkt.loads(search_geom_str)
    target_geom = wkt.loads(target_geom_str)
    doTheyRelate = search_geom.intersects(target_geom)
    return doTheyRelate

In [9]:
search_wkt = yosemite['WKT'][0]  #there is only one polygon in this table
target_wkt = calcounties.where(calcounties['NAME'] == 'Madera County')['WKT'][0]  #this is the polygon for Madera County
areTheyNeighbors(search_wkt,target_wkt)

True

That's great! We have taken the Yosemite polygon and compared it to one county polygon and found that they intersect. We can use the **apply** method of Tables to check every county geometry using the areTheyNeighbors function. This function will return True if the polygons intersect, else false. The Table **where** method will then select those rows of the calcounties only where areTheyNeighbors is true. See it in action below.

In [1]:
#find all the counties that intersect yosemite
myneighbor_table = calcounties.where(calcounties.apply(lambda x: areTheyNeighbors(search_wkt,x), ['WKT']))
myneighbor_table

NameError: name 'calcounties' is not defined

Now let's plot Yosemite and the counties in which it is located to make sure our results are correct.  We will use **folium** to map the polygons. Folium accepts polygon data in geojson format so we create a function to convert an array of WKT geometries to geojson.

In [11]:
# A function to take an array of WKT geom (e.g. from A Table Column) and return a geojson string
def make_geojson_from_wktArray(my_geom):
    head = '{"type":"FeatureCollection","features":['
    tail = ']}'

    numfeatures = len(my_geom)
    
    if numfeatures == 1:
        feature_geom = ogr.CreateGeometryFromWkt(my_geom[0]) # ogr polygon
        geojson_str = feature_geom.ExportToJson()
        return geojson_str
    
    else:
        geojson_str = head
        counter = 1
        for r in my_geom:
            feature_geom = ogr.CreateGeometryFromWkt(r) # ogr polygon
            geojson_str += feature_geom.ExportToJson()
            if counter < numfeatures:
                geojson_str += ","
                
            counter=counter +1

        geojson_str += tail
        return geojson_str

Now we use the function to create geojson strings.

In [12]:
search_geojson = make_geojson_from_wktArray(yosemite['WKT']) #convert geometries to geojson

neighbor_geojson = make_geojson_from_wktArray(myneighbor_table['WKT']) # convertgeometries to geojson


We can now map the geojson. First we take the center point, or centroid, of our search geometry to center the map.

In [24]:
# Center our map on yosemite
ctr_lon = wkt.loads(yosemite['WKT'][0]).centroid.x
ctr_lat = wkt.loads(yosemite['WKT'][0]).centroid.y

neighbor_map = folium.Map(zoom_start=8, location=[ctr_lat, ctr_lon])

neighbor_map.geo_json(geo_str=neighbor_geojson)
neighbor_map.geo_json(geo_str=search_geojson, fill_color='red')

neighbor_map # draw map

/Users/patty/anaconda3/lib/python3.4/site-packages/folium/folium.py:504: UserWarning: This method is deprecated. Please use Map.choropleth instead.
  warnings.warn('This method is deprecated. '


## Part 3. Calculating Area of Overlap

The map confirms the spatial relationships between Yosemite and the counties in the myneighbor_counties table. We can see that the park is not evenly divided among all the counties. Let's calculate how much of the park is in each county. 

We did all of the above topological queries with data in geographic coordinates. This is ok for examining topological relationships but not for operations that require metric calculations. So, we will need to transform, or project, the counties to an appropriate 2D coordinate reference system in order to calculate area. We will use **pyproj** to create a projection transformation object and then apply it to the shapely geometries. This is shown below.

First a little code that shows how to make a pyproj CRS transformation object.

In [14]:
from functools import partial
import pyproj
from shapely.ops import transform

transformTo3857 = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),  # source coordinate system - WGS 84, EPSG:4326
    pyproj.Proj(init='epsg:3857'))  # destination coordinate system - Web Mercator, EPSG: 3857

#g2 = transform(transformTo3857, g1)        # apply CRS transformation to data

In this next cell we show how to get the area of overlap for two geometeries in WKT format.

In [15]:
search_wkt = yosemite['WKT'][0]  # There is only one polygon in the Yosemite table
target_wkt = calcounties.where(calcounties['NAME'] == 'Madera County')['WKT'][0]  #Cal Counties

search_geom = wkt.loads(search_wkt) # convert WKT to a shapely geometry
target_geom = wkt.loads(target_wkt)


search_geom = transform(transformTo3857,search_geom) # Transform the geometry to a new CRS - EPSG:3857, web mercator
target_geom = transform(transformTo3857,target_geom)
 
area_olap = search_geom.intersection(target_geom).area
area_olap # in units of CRS
area_olap / (1000 * 1000)  # in sq km

498.73959847613895

## Question 3
Using the code above as reference, 
- Write a function to compute the area of overlap in KM for all the geometeries in the myneighbor_table and Yosemite.
- You will need to define a different transformation function because Web Mercator (EPSG:3857) is not appropriate for area calculations.
- Apply your area of overlap function to the myneighbors table.
- Show your work in the code cell below.

In [16]:
# Question 3 answer here


WKT,STATEFP,COUNTYFP,NAME,ALAND,AWATER,area_overlap
"MULTIPOLYGON (((-120.321526 37.5244,-120.38767 37.633364 ...",6,43,Mariposa County,3752425638,36268140,982.072
"MULTIPOLYGON (((-119.308904 38.007149,-119.307034 38.012 ...",6,51,Mono County,7896722221,214695416,1.42782
"MULTIPOLYGON (((-120.500161 38.004124,-120.499822 38.007 ...",6,109,Tuolumne County,5752116633,138684399,1720.4
"MULTIPOLYGON (((-120.106385 37.167153,-120.094743 37.172 ...",6,39,Madera County,5535058234,41957135,312.512


## Part 4. Getting Touchy

Let's explore the spatial relationshop **touches**. We want to see what California Counties share a border with Nevada.  First let's map the polygons for both the Nevada and California county polygons.

## Question 4

Use the code from above to create a map of Nevada and the California counties.
- Center the map on Nevada by extracting its centroid.
- Update the zoom level so that is appropriate for showing California and Nevada.


In [26]:
# Your Question 4 Answer here


/Users/patty/anaconda3/lib/python3.4/site-packages/folium/folium.py:504: UserWarning: This method is deprecated. Please use Map.choropleth instead.
  warnings.warn('This method is deprecated. '


## Question 5

Expanding from the code above used to create the myneighbors table of California Counties in which Yosemite is located, create a neighbors table of California counties that border (or touch) Nevada.  A simple way to do this is to copy the **areTheyNeighbors** function and change the spatial comparison operator. Alternatively, you could parameterize that function to take an argument, e.g. relation="intersects".  Show your work in the cell below testing both the **touches** and the **intersects** spatial comparison operators.

In [18]:
# Question 5 - your Answer in this cell


WKT,STATEFP,COUNTYFP,NAME,ALAND,AWATER
"MULTIPOLYGON (((-121.32191 40.654957,-121.319976 40.9058 ...",6,35,Lassen County,11761622251,463440332
"MULTIPOLYGON (((-118.337579 36.654801,-118.335627 36.661 ...",6,27,Inyo County,26368506526,119060126
"MULTIPOLYGON (((-119.308904 38.007149,-119.307034 38.012 ...",6,51,Mono County,7896722221,214695416
"MULTIPOLYGON (((-120.073331 38.70109,-119.964948 38.7759 ...",6,3,Alpine County,1912243146,12557256
"MULTIPOLYGON (((-121.118617 38.717118,-121.118696 38.715 ...",6,17,El Dorado County,4423442486,203356347


## Question 6 - Add your answer to this cell by double clicking on it and typing.

- How many neigbors do you get with the intersect operator?
- How many with the touch operator?
- How many did you expect?
- How do you explain these differences?



## Spatial Relationships are Complicated!

As you likely saw above from your responses to Questions 4-6, spatial relationship queries are complicated. They don't always yield expected results. Many such relationships are difficult to test if the geometries, particularly from different data sets, do not align.

## Question 7

Write a function that buffers the search geometry before testing if two geometries intersect. The function should take a buffer_distance parameter that is in meters. That means you will need to transform the geometries from WGS84 to once suitable for these data. An appropriate buffer distance is one that is large enough to intersect with bordering counties but not so large as to include other counties. Apply the function to query the california counties that are along the Nevada border. Input your results in the cell below. Your code should show the buffer distance that identifies all of the counties along the border.

In [19]:
# Question 7 - Input your answer here



WKT,STATEFP,COUNTYFP,NAME,ALAND,AWATER
"MULTIPOLYGON (((-120.655585 39.69356,-120.654227 39.7066 ...",6,91,Sierra County,2468686345,23299112
"MULTIPOLYGON (((-121.065436 39.006533,-121.061653 39.010 ...",6,61,Placer County,3644346108,246376805
"MULTIPOLYGON (((-121.32191 40.654957,-121.319976 40.9058 ...",6,35,Lassen County,11761622251,463440332
"MULTIPOLYGON (((-118.337579 36.654801,-118.335627 36.661 ...",6,27,Inyo County,26368506526,119060126
"MULTIPOLYGON (((-119.308904 38.007149,-119.307034 38.012 ...",6,51,Mono County,7896722221,214695416
"MULTIPOLYGON (((-117.667244 34.734334,-117.667292 34.822 ...",6,71,San Bernardino County,51947497395,123929658
"MULTIPOLYGON (((-120.073331 38.70109,-119.964948 38.7759 ...",6,3,Alpine County,1912243146,12557256
"MULTIPOLYGON (((-121.118617 38.717118,-121.118696 38.715 ...",6,17,El Dorado County,4423442486,203356347
"MULTIPOLYGON (((-121.448902 41.472813,-121.448981 41.776 ...",6,49,Modoc County,10140841147,745929078
"MULTIPOLYGON (((-120.713744 39.482788,-120.717024 39.489 ...",6,57,Nevada County,2480638586,41513176


# The End

Last updated 2/28/2016 by pfrontiera@berkeley.edu